In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from openai import OpenAI

In [ ]:
client = OpenAI(api_key="your_key")


In [3]:
train_df = pd.read_csv(r"C:\\Users\\lu17w\\Downloads\\job_data_files\\work_arrangements_development_set.csv")  

In [4]:
def row_to_text(row, include_label=True):
    base = (
        f"job_ad: {row['job_ad']}\n"
        # f"work arrangement: {row['y_true']}"
    )
    return base + (f"\nLabel: y_true = {row['y_true']}" if include_label else "")

In [5]:
shots = "\n\n".join([f"example{i+1}：\n{row_to_text(train_df.iloc[i])}" for i in range(10)])

In [6]:
test_df = pd.read_csv("C:\\Users\\lu17w\\Downloads\\job_data_files\\work_arrangements_test_set.csv")

test_df["predicted"] = ""
test_df["true_label"] = test_df["y_true"].astype(str)
results = []
correct = 0

for i, row in test_df.iterrows():
    query = f"query：\n{row_to_text(row, include_label=False)}\nyour_predict：y_true ="

    user_prompt = (
        "You are a classification assistant. Below are some job descriptions along with their corresponding work arrangement (y_true)：\n\n"
        "Please only give me answer, no more other things,you should choose a word in OnSite, Remote and Hybrid as answer.)"
        # f"{shots}\n\n{query}"
        f"{query}"
    )

    messages = [
        {"role": "user", "content": user_prompt}
    ]

    response = client.chat.completions.create(
        model="ft:gpt-4.1-mini-2025-04-14:personal:6713:BQuQ64ON",
        # model='gpt-4.1-mini',
        messages=messages,
        max_tokens=30,
        temperature=0.7
    )

    prediction = response.choices[0].message.content.strip().split("\n")[0]


    results.append({
        "index": i,
        "predicted": prediction,
        "true_label": str(row["y_true"])
    })
    if prediction == str(row["y_true"]):
        correct += 1
    if (i + 1) % 100 == 0:
        checkpoint_df = pd.DataFrame(results)
        filename = f"gpt4mini/gpt-4.1-mini-work_predictions_zeroshot.csv"
        checkpoint_df.to_csv(filename, index=False)
        print(f"{i+1}, save result to：{filename}")
    number = i+1

filename = f"gpt4mini/gpt-4.1-mini-work_predictions_zeroshot.csv"
final_df = pd.DataFrame(results)
final_df.to_csv(filename, index=False)

In [7]:
accuracy = correct / number
print(f"accuracy：{accuracy:.2%}")
print(f"save result to：{filename}")


accuracy：94.95%
save result to：gpt4mini/gpt-4.1-mini-work_predictions_zeroshot.csv
